In [2]:
!pip install  -U -q trl peft math_verify
!pip install unsloth-zoo==2025.9.9
# Tested with transformers==4.47.1, trl==0.14.0, datasets==3.2.0, peft==0.14.0, accelerate==1.2.1, math_verify==0.3.3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.9.9 requires transformers!=4.52.0,!=4.52.1,!=4.52.2,!=4.52.3,!=4.53.0,!=4.54.0,!=4.55.0,!=4.55.1,<=4.55.4,>=4.51.3, but you have transformers 4.57.3 which is incompatible.
  Using cached transformers-4.55.4-py3-none-any.whl.metadata (41 kB)
  Using cached tokenizers-0.21.4-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
INFO: pip is looking at multiple versions of trl to determine which version is compatible with other requirements. This could take a while.
  Using cached trl-0.26.1-py3-none-any.whl.metadata (11 kB)
  Using cached trl-0.26.0-py3-none-any.whl.metadata (11 kB)
  Using cached trl-0.25.1-py3-none-any.whl.metadata (11 kB)
  Using cached trl-0.25.0-py3-none-any.whl.metadata (11 kB)
  Using cached trl-0.24.0-py3-none-any.whl.metadata (11 kB)
  Using cached trl-

In [2]:
pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [ipywidgets]
Note: you may need to restart the kernel to use updated packages.


In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [99]:
from datasets import load_dataset

dataset_id = 'AI-MO/NuminaMath-TIR'
train_dataset, test_dataset = load_dataset(dataset_id, split=['train[:10%]', 'test[:10%]'])

In [92]:
print(train_dataset["solution"][0])

To determine the coefficient of \(x^2y^6\) in the expansion of \(\left(\frac{3}{5}x - \frac{y}{2}\right)^8\), we can use the binomial theorem.

The binomial theorem states:
\[
(a + b)^n = \sum_{k=0}^{n} \binom{n}{k} a^{n-k} b^k
\]

In this case, \(a = \frac{3}{5}x\), \(b = -\frac{y}{2}\), and \(n = 8\).

We are interested in the term that contains \(x^2y^6\). In the general term of the binomial expansion:
\[
\binom{8}{k} \left(\frac{3}{5}x\right)^{8-k} \left(-\frac{y}{2}\right)^k
\]

To get \(x^2\), we need \(8 - k = 2\), thus \(k = 6\).

Substituting \(k = 6\) into the expression:
\[
\binom{8}{6} \left(\frac{3}{5}x\right)^{8-6} \left(-\frac{y}{2}\right)^6 = \binom{8}{6} \left(\frac{3}{5}x\right)^2 \left(-\frac{y}{2}\right)^6
\]

Now, we will compute each part of this expression.

1. Calculate the binomial coefficient \(\binom{8}{6}\).
2. Compute \(\left(\frac{3}{5}\right)^2\).
3. Compute \(\left(-\frac{y}{2}\right)^6\).
4. Combine everything together to get the coefficient of \(x^2y^6

In [100]:
SYSTEM_PROMPT = (
    "A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant "
    "first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning "
    "process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., "
    "<think> reasoning process here </think><answer> answer here </answer>"
)

def make_conversation(example):
    return {
        "prompt": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": example["problem"]},
        ],
    }

train_dataset = train_dataset.map(make_conversation)
test_dataset = test_dataset.map(make_conversation)

Map:   0%|          | 0/7244 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [101]:
print(train_dataset[0]['prompt'])

[{'content': 'A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think><answer> answer here </answer>', 'role': 'system'}, {'content': 'What is the coefficient of $x^2y^6$ in the expansion of $\\left(\\frac{3}{5}x-\\frac{y}{2}\\right)^8$?  Express your answer as a common fraction.', 'role': 'user'}]


In [102]:
train_dataset = train_dataset.remove_columns(['messages', 'problem'])
print(train_dataset)

Dataset({
    features: ['solution', 'prompt'],
    num_rows: 7244
})


In [103]:
import torch
from transformers import AutoModelForCausalLM

model_id = "Qwen/Qwen2-0.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="auto",
)

In [104]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"],
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

trainable params: 540,672 || all params: 494,573,440 || trainable%: 0.1093


In [105]:
import re
def format_reward(completions, **kwargs):
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<think>.*?</think>\s*<answer>.*?</answer>$"
    completion_contents = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, content) for content in completion_contents]
    rewards_list = [1.0 if match else 0.0 for match in matches]
    return [1.0 if match else 0.0 for match in matches]

In [106]:
from math_verify import LatexExtractionConfig, parse, verify
def accuracy_reward(completions, **kwargs):
    """Reward function that checks if the completion is the same as the ground truth."""
    solutions = kwargs['solution']
    completion_contents = [completion[0]["content"] for completion in completions]
    rewards = []
    for content, solution in zip(completion_contents, solutions):
        gold_parsed = parse(solution, extraction_mode="first_match", extraction_config=[LatexExtractionConfig()])
        answer_parsed = parse(content, extraction_mode="first_match", extraction_config=[LatexExtractionConfig()])
        if len(gold_parsed) != 0:
            try:
                rewards.append(float(verify(answer_parsed, gold_parsed)))
            except Exception:
                rewards.append(0.0)
        else:
            rewards.append(1.0)
    return rewards

In [112]:
from trl import GRPOConfig

# Configure training arguments using GRPOConfig
training_args = GRPOConfig(
    output_dir="Qwen2-0.5B-GRPO-test-2",
    learning_rate=1e-3,
    remove_unused_columns=False, # to access the solution column in accuracy_reward
    gradient_accumulation_steps=16,
    num_train_epochs=1,
    bf16=True,

    # Parameters that control de data preprocessing
    max_completion_length=64, # default: 256
    num_generations=2, # default: 8
    max_prompt_length=512, # default: 512

    # Parameters related to reporting and saving
    report_to=["tensorboard"],
    logging_steps=1,
    push_to_hub=True,
    save_strategy="steps",
    save_steps=1,
)

In [113]:
from trl import GRPOTrainer

trainer = GRPOTrainer(
    model=model,
    reward_funcs=[format_reward, accuracy_reward],
    args=training_args,
    train_dataset=train_dataset
)
# trainer.train()

In [ ]:
trainer.train()

Step,Training Loss
1,0.000000
2,0.000000


KeyboardInterrupt: 

In [ ]:
logs = trainer.state.log_history

In [ ]:
trainer.save_model(training_args.output_dir)
trainer.push_to_hub(dataset_name=dataset_id)

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...ut.tfevents.1767959057.aaids.6021.0: 100%|##########| 6.83kB / 6.83kB            

  ...t.tfevents.1767958158.aaids.68176.0: 100%|##########| 22.6kB / 22.6kB            

  ...ut.tfevents.1767959151.aaids.6021.1: 100%|##########| 98.2kB / 98.2kB            

...

  ...t.tfevents.1767958158.aaids.68176.0: 100%|##########| 22.6kB / 22.6kB            

  ...ut.tfevents.1767959151.aaids.6021.1: 100%|##########| 98.2kB / 98.2kB            

  ...PO-test-1/adapter_model.safetensors: 100%|##########| 2.18MB / 2.18MB            

  ...en2-0.5B-GRPO-test-1/tokenizer.json: 100%|##########| 11.4MB / 11.4MB            

CommitInfo(commit_url='https://huggingface.co/SulemanSahib/Qwen2-0.5B-GRPO-test-1/commit/dd87abb80ef17cf32108bd3114762709f6b71f25', commit_message='End of training', commit_description='', oid='dd87abb80ef17cf32108bd3114762709f6b71f25', pr_url=None, repo_url=RepoUrl('https://huggingface.co/SulemanSahib/Qwen2-0.5B-GRPO-test-1', endpoint='https://huggingface.co', repo_type='model', repo_id='SulemanSahib/Qwen2-0.5B-GRPO-test-1'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import AutoTokenizer

model_id = "SulemanSahib/Qwen2-0.5B-GRPO-test-1" #"Qwen/Qwen2-0.5B-Instruct" # 
trained_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="auto",
)
trained_tokenizer = AutoTokenizer.from_pretrained(model_id)

In [82]:
print(test_dataset['prompt'][3])
print(test_dataset['solution'][3])

[{'content': 'A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think><answer> answer here </answer>', 'role': 'system'}, {'content': 'If each exterior angle of a regular polygon is $30^{\\circ}$, then the number of sides of this polygon is:\n\nA: $6$\n\nB: $8$\n\nC: $9$\n\nD: $12$', 'role': 'user'}]
To solve the problem of finding the number of sides of a regular polygon given that each exterior angle is \(30^\circ\), we can use the property that the sum of the exterior angles of any polygon is always \(360^\circ\). Therefore, if each exterior angle is \(30^\circ\), we can determine the number of sides of the polygon by dividing the total sum of the exterior angles by the measure

In [70]:
import time

def generate_with_reasoning(prompt):
  # Build the prompt from the dataset
  prompt = " ".join(entry['content'] for entry in prompt)

  # Tokenize and move to the same device as the model
  inputs = trained_tokenizer(prompt, return_tensors="pt").to(trained_model.device)

  # Generate text without gradients
  start_time = time.time()
  with torch.no_grad():
      output_ids = trained_model.generate(**inputs, max_length=500)
  end_time = time.time()

  # Decode and extract model response
  generated_text = trained_tokenizer.decode(output_ids[0], skip_special_tokens=True)

  # Get inference time
  inference_duration = end_time - start_time

  # Get number of generated tokens
  num_input_tokens = inputs['input_ids'].shape[1]
  num_generated_tokens = output_ids.shape[1] - num_input_tokens

  return generated_text, inference_duration, num_generated_tokens

In [89]:
prompt = test_dataset['prompt'][3]
generated_text, inference_duration, num_generated_tokens = generate_with_reasoning(prompt)
print(generated_text)

A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think><answer> answer here </answer> If each exterior angle of a regular polygon is $30^{\circ}$, then the number of sides of this polygon is:

A: $6$

B: $8$

C: $9$

D: $12$ 

The reason I chose D: Because if you look at an equilateral triangle, all the angles are equal to 60 degrees.

I think the correct answer should be D: 12. However, the reasoning behind my choice was that because if you look at an equilateral triangle, all the angles are equal to 60 degrees.
I know this might not be right but I am just curious as to why the other answers were wrong. Any thoughts would be greatly appreciated.


In [74]:
train_dataset['prompt'][3]

[{'content': 'A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think><answer> answer here </answer>',
  'role': 'system'},
 {'content': 'Find the smallest positive $a$ such that $a$ is a multiple of $4$ and $a$ is a multiple of $14.$',
  'role': 'user'}]

In [35]:
print(f"Inference time: {inference_duration:.2f} seconds")
print(f"Generated tokens: {num_generated_tokens}")

Inference time: 0.16 seconds
Generated tokens: 20


In [90]:
prompt_text = " ".join(entry['content'] for entry in prompt)
response_text = generated_text[len(prompt_text):].strip()
print(response_text)

The reason I chose D: Because if you look at an equilateral triangle, all the angles are equal to 60 degrees.

I think the correct answer should be D: 12. However, the reasoning behind my choice was that because if you look at an equilateral triangle, all the angles are equal to 60 degrees.
I know this might not be right but I am just curious as to why the other answers were wrong. Any thoughts would be greatly appreciated.
